In [ ]:
import json

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from collections import Counter

In [ ]:
data = [json.loads(line) for line in open('Datasets/SST/data/dev.jsonl')]

In [ ]:
labels = [d['label'] for d in data]

In [ ]:
Counter(labels)

In [ ]:
lengths_r = np.array([len(d['rationales']) for d in tao_rationale])
lengths_d = np.array([len(d['metadata']['tokens']) for d in rationales])

In [ ]:
name = {'top_k' : 'top k', 'max_length' : 'top k contiguous'}
for saliency in ['wrapper'] : #, 'simple_gradient', 'integrated_gradient'] :
    for rat in ['top_k', 'max_length'] :
        ps = []
        acc = []
        for p in [20, 40, 60, 80, 100] :
            rationales = [json.loads(line) 
                          for line in open(f'outputs/bert_classification/SST/bert_base_uncased/{saliency}_saliency/{rat}_rationale/MAX_LENGTH_PERCENT={p}/dev.jsonl')]
            lengths_d = np.array([len(x['document'].split()) for x in data[:len(rationales)]])
            lengths_r = np.array([len(x['document'].split()) for x in rationales])
            metrics = json.load(open(f'outputs/bert_classification/SST/bert_base_uncased/{saliency}_saliency/{rat}_rationale/MAX_LENGTH_PERCENT={p}/model_b/metrics.json'))
            print(np.mean(lengths_r/lengths_d), metrics['best_validation_accuracy'])
            ps.append(np.mean(lengths_r/lengths_d))
            acc.append(metrics['best_validation_accuracy'])
        plt.plot([0] + ps, [0.509] + acc, label= '[CLS] Attention +' + name[rat])
        
ps , acc = [], []
for r in [0.2, 0.4, 0.6, 0.8, 1.0] :
    metrics = json.load(open(f'outputs/bert_encoder_generator/SST/run_1_enc_gen/{r}/metrics.json'))
    ps.append(metrics['best_validation__rat_length'])
    acc.append(metrics['best_validation_accuracy'])
    
plt.scatter(ps, acc, label='Tao')
        
plt.xlabel('Percentage of Document Selected')
plt.ylabel('Accuracy')
plt.tight_layout()
plt.legend()
plt.savefig('sst-max-length-length.pdf')

In [ ]:
from allennlp.data.token_indexers import PretrainedBertIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary

In [ ]:
index = PretrainedBertIndexer(do_lowercase=True, max_pieces=100, pretrained_model='bert-base-uncased', truncate_long_sequences=False)

In [ ]:
vocab = Vocabulary()

In [ ]:
tokens = [Token(str(i)) for i in range(101)]

In [ ]:
index.tokens_to_indices(tokens, vocab, index_name='bert')

In [ ]:
from pytorch_transformers import BertTokenizer, BertForTokenClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
labels = torch.tensor([1] * input_ids.size(1)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, labels=labels)
loss, scores = outputs[:2]